In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.
C:\Users\suwarna\Anaconda3\envs\pytorch\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\suwarna\Anaconda3\envs\pytorch\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\suwarna\Anaconda3\envs\pytorch\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\suwarna\Anaconda3\envs\pyt

In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [3]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)
y_train = y_data[:1395]
y_test = y_data[1395:]

def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(128, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(128, return_sequences = True))
    model.add(SimpleRNN(128, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))


Epoch 1/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.5634 - accuracy: 0.6602
Epoch 2/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.1704 - accuracy: 0.7147: 0s - loss: 1.190
Epoch 3/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.1372 - accuracy: 0.7104
Epoch 4/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.1151 - accuracy: 0.7161
Epoch 5/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.1025 - accuracy: 0.7133
Epoch 6/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.0816 - accuracy: 0.7183
Epoch 7/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.0686 - accuracy: 0.7111
Epoch 8/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.0647 - accuracy: 0.7032
Epoch 9/200
1395/1395 [==============================] - 3s 2ms/step - loss: 1.0521 - accuracy: 0.7197: 
Epoch 10/200
1395/1395 [============================

In [4]:
X_train[0]

array([[   1],
       [ 245],
       [ 273],
       [ 207],
       [ 156],
       [  53],
       [  74],
       [ 160],
       [  26],
       [  14],
       [  46],
       [ 296],
       [  26],
       [  39],
       [  74],
       [2979],
       [3554],
       [  14],
       [  46],
       [4689],
       [4329],
       [  86],
       [  61],
       [3499],
       [4795],
       [  14],
       [  61],
       [ 451],
       [4329],
       [  17],
       [  12],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0]])

In [5]:
np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1)).shape

(1395, 49, 1)

In [6]:
y_pred

array([ 3,  4,  1,  3,  3,  4,  3,  1,  3,  3,  3,  3,  1,  3,  3,  4,  3,
        3,  3,  3,  3,  4,  3,  3,  3, 20,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,
       36,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  1,
        3,  3,  2,  3,  3,  3,  3,  4,  3, 19,  4,  4, 19,  3,  3,  4,  3,
        1,  3, 17,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 17,  4,
        3,  3,  4,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,
        3,  3,  3,  3,  3, 24,  3, 16,  3,  3,  3,  3,  3,  4,  3,  3,  3,
        4,  3,  3,  3,  3,  3,  3,  3,  4,  3,  4,  3,  1,  3, 24,  3,  3,
        3,  3,  3,  4,  4,  3, 45,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  8,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 20,  3, 19,
        3,  3,  3,  3,  1,  3,  3,  3, 13,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  3,  4, 11,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 16,  3,
       19,  3,  3,  3,  3